In [1]:
from bioio import BioImage
import numpy as np
import os
from glob import glob
import skimage as sk
import napari
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt
from natsort import natsorted



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.8.0+cu126! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [2]:
test_file = '../WT_RAB5GFP_RAB7mCH/WT_RAB5GFP_RAB7mCH_Image_5_Airyscan_Processing.czi'
czi_file = BioImage(test_file)
img = czi_file.data
img = np.squeeze(img)


In [11]:
img.shape

(2, 71, 1000, 1000)

In [22]:
ch1_img = np.squeeze(ch1_img)

In [7]:
viewer = napari.view_image(img, scale=[czi_file.physical_pixel_sizes.Z,czi_file.physical_pixel_sizes.X,czi_file.physical_pixel_sizes.Y])

C:\Users\Kristin.Gallik\AppData\Local\Temp\ipykernel_5948\1044138916.py:1: FutureWarning: `napari.view_image` is deprecated and will be removed in napari 0.7.0.
Use `viewer = napari.Viewer(); viewer.add_image(...)` instead.
  viewer = napari.view_image(img, scale=[czi_file.physical_pixel_sizes.Z,czi_file.physical_pixel_sizes.X,czi_file.physical_pixel_sizes.Y])


In [3]:
if core.use_gpu()==False:
  raise ImportError("No GPU access, change your runtime")

model = models.CellposeModel(gpu=True)

In [ ]:
first_channel = 0
second_channel = 1
selected_channels = []
for i, c in enumerate([first_channel, second_channel]):
  if c == 'None':
    continue
  if int(c) > img.shape[0]:
    assert False, 'invalid channel index, must have index greater or equal to the number of channels'
  if c != 'None':
    selected_channels.append(int(c))



img_selected_channels = np.zeros_like(img)
img_selected_channels[:, :, :len(selected_channels)] = img[:, :, selected_channels]

io.logger_setup()
flow_threshold = 0.4
cellprob_threshold = 0.0
tile_norm_blocksize = 0

masks, flows, styles = model.eval(img_selected_channels, channel_axis=0, z_axis=1, batch_size=32, flow_threshold=flow_threshold, cellprob_threshold=cellprob_threshold,
                                  normalize={"tile_norm_blocksize": tile_norm_blocksize},do_3D=True)

fig = plt.figure(figsize=(12,5))
plot.show_segmentation(fig, img_selected_channels, masks, flows[0])
plt.tight_layout()
plt.show()

2025-08-16 11:41:40,924 [INFO] WRITING LOG OUTPUT TO C:\Users\Kristin.Gallik\.cellpose\run.log
2025-08-16 11:41:40,926 [INFO] 
cellpose version: 	4.0.6 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.8.0+cu126
2025-08-16 11:41:42,993 [INFO] running YX: 71 planes of size (1000, 1000)
2025-08-16 11:52:06,091 [INFO] 4%|4         | 3/71 [10:23<3:55:23, 207.70s/it]
